In [1]:
import requests
from kiteconnect import KiteConnect
import datetime
import os
import time
from tqdm import tqdm
import pandas as pd

In [2]:
nifty_50_symbols = []

with open("nifty50.txt", "r") as f:
    for line in f:
        nifty_50_symbols.append(line.split()[0].strip().lower())

In [3]:
kite = KiteConnect(api_key="cfafsza8jnlnwbk2")

In [4]:
kite.login_url()

'https://kite.zerodha.com/connect/login?api_key=cfafsza8jnlnwbk2&v=3'

In [5]:
data = kite.generate_session("onBLZJeW8y5ksUHxkPilAYpcmc0X2luF", api_secret="tc21c8g87cwpsxcdijo3sn281pc3wdme")
kite.set_access_token(data["access_token"])

In [6]:
instruments = kite.instruments(exchange= kite.EXCHANGE_BSE)

In [7]:
nifty_50_instruments = []

for instrument in instruments:
    if instrument["tradingsymbol"].strip().lower() in nifty_50_symbols:
        nifty_50_instruments.append(instrument)

In [8]:
def getDataFromInstrument(from_date,to_date,interval,instrument):
    delta = datetime.timedelta(days= 15)
    
    _from_date = from_date
    data = []
    while _from_date <= to_date:
        _to_date = min(to_date, _from_date + delta)
        try:
            temp_data = kite.historical_data(instrument_token= instrument["instrument_token"], from_date=  _from_date,
                                to_date= _to_date,interval= interval
                                )   
            data.extend(temp_data)    
            _from_date += delta
        except Exception as e:
            print(e)
            continue

        time.sleep(.5)

    for d in data:
        d["date"] = d["date"].replace(tzinfo = None)
    return data


def getCSVFromInstrument(instrument,start_date = datetime.datetime(year= 2010, month= 1,day= 1),
                         end_date = datetime.datetime(year= 2024, month= 6,day= 1),
                         intervals = ["minute","5minute", "day"],
                         base_path = "/home/captanlevi/Desktop/StockOverflow/data"
                         ):
    
    
    for interval in intervals:
        path = os.path.join(base_path, interval,instrument["tradingsymbol"] + ".csv")
        #print(path)
        if os.path.exists(path):
            continue

        data = getDataFromInstrument(from_date= start_date, to_date= end_date, interval= interval, instrument= instrument)
        data = pd.DataFrame(data)
        data.set_index("date", inplace= True)
        data.sort_index(inplace= True)
        
        data.to_csv(path)
        
    
    

In [10]:
for instrument in nifty_50_instruments:
    getCSVFromInstrument(instrument= instrument)

Unknown Content-Type (text/html) with response: (b'<html>\r\n<head><title>502 Bad Gateway</title></head>\r\n<body>\r\n<center><h1>502 Bad Gateway</h1></center>\r\n</body>\r\n</html>\r\n')


KeyboardInterrupt: 